In [5]:
import pandas as pd
import numpy as np
import nanoid
import pymongo
from openai import OpenAI
import requests
from datetime import datetime



In [15]:
quantites = np.random.randint(1,100,size=48)
prices_buy = np.random.randint(1,2,size=48)*10 + np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1
prices_sale = prices_buy - np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1


In [18]:
data = pd.read_csv('data/flowershopdata.csv')
data['Description'] = data[['Description', 'Needs','Season']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
data['Buy_price'] = prices_buy
data['Sale_price'] = np.round(prices_sale,2)
data['Quantities'] = quantites
jpg_paths = [f"init_data/products/img_flowers/{flower.lower()}.jpg" for flower in data['Name']]
data['pic_path'] = jpg_paths
data['id'] = [nanoid.generate(size=10) for x in range(48)]
data['date'] =[datetime.now().strftime("%d-%m-%Y %H:%M:%S") for x in range(48)]
data['tags'] = ['flower' for x in range(48)]
data.drop(columns=['Needs','Season'],inplace=True)

In [19]:
data.to_csv('data/flowershopdata_clean.csv')

In [46]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def fetch_first_image_url(query):
    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    search_url = f"https://pl.pinterest.com/search/pins/?q={query}%20flower&rs=typed"
    driver.get(search_url)

    time.sleep(2)
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')

    if len(image_elements) >= 10:
        # Get the URL of the 10th image
        image_element = image_elements[9]  # Index 9 corresponds to the 10th image (0-based index)
        image_url = image_element.get_attribute('src')
    else:
        print("Not enough images found on the page")
        image_url = None

    driver.quit()
    return image_url

def save_image_from_url(url, directory, filename):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image_path = os.path.join(directory, filename)
    image.save(image_path)
    print(f"Image saved to {image_path}")



In [6]:
import nanoid
def create_product(product_data):
    base_url = "http://127.0.0.1:8000"
    response = requests.post(f"{base_url}/products/", json=product_data)
    print(response.json())

In [7]:
data = pd.read_csv('data/flowershopdata_clean.csv').drop(columns=['Unnamed: 0'])



In [40]:
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
print(names)

[]


In [32]:
# from sentence_transformers import SentenceTransformer, util
from qdrant_client import QdrantClient, AsyncQdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct

# model = SentenceTransformer('intfloat/e5-small-v2')
vec_len = 384
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
if 'products_description' not in names:
    client.create_collection(
        collection_name="products_description",
        vectors_config=VectorParams(size=vec_len, distance=Distance.COSINE),)
else:
    print('Collection exits')


In [39]:
client.delete_collection("products_description")

False

In [8]:
points = [PointStruct(id=index, 
                      vector=model.encode(item['Description'], normalize_embeddings=True), 
                      payload={"Id": item['id'],
                               "Name": item['Name'],
                               "Description":item['Description']}) for index,item in data.iterrows()]


In [215]:
operation_info = client.upsert(
    collection_name="products_description",
    wait=True,
    points=points,
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [3]:
from pymongo import MongoClient
from openai import OpenAI
from typing import List
# from sentence_transformers import SentenceTransformer
from qdrant_client import AsyncQdrantClient, models


In [88]:
question = "I would like white flower that needs acidic soil"
# model = SentenceTransformer("intfloat/e5-small-v2")
bot = {
    "mongo_client": MongoClient("mongodb://localhost:27017/"),
    "qdrant_client": QdrantClient("http://localhost:6333"),
    "openai_client":  OpenAI(api_key="sk-proj-8j1NyU5gvA4njucbGG3uT3BlbkFJxBLHcRn4z7futfYb4sgb")
},

In [42]:
client = AsyncQdrantClient("http://localhost:6333")

In [68]:
sad = await client.scroll(
    collection_name ="products_description",
    scroll_filter = models.Filter(
        must = [models.FieldCondition(
            key='Name',
            match =models.MatchValue(
                value= 'Bloodroot'))
                ]
    ),
    with_vectors = True
)

sad[0][0].vector

[-0.07306243,
 0.026809188,
 0.017532839,
 0.027319789,
 0.016868953,
 -0.014223786,
 0.100838155,
 -0.040248465,
 -0.008592019,
 0.050778165,
 0.033833582,
 -0.035047084,
 -0.050870217,
 0.058055907,
 0.02894813,
 -0.0115545485,
 -0.034937903,
 0.039293673,
 -0.09408304,
 0.06397073,
 0.106469914,
 -0.046439257,
 -0.00390669,
 -0.06518312,
 -0.04733827,
 -0.0125166215,
 0.060824092,
 0.05743405,
 -0.072144106,
 -0.1348871,
 -0.048684217,
 0.04071604,
 -0.004059148,
 -0.03329434,
 0.061386973,
 -0.0427294,
 -0.022997458,
 0.053379614,
 0.013719811,
 0.057668895,
 -0.048779767,
 -0.07621413,
 0.030835312,
 -0.0139691755,
 -0.009902468,
 0.011785127,
 -0.04697574,
 -0.020275576,
 0.06230306,
 0.013810104,
 -0.041896723,
 -0.02136755,
 0.035126023,
 0.07269824,
 0.049637813,
 0.02645026,
 0.07165351,
 0.047983,
 0.07585909,
 0.068684965,
 0.007475001,
 0.046368644,
 -0.11471148,
 0.102662764,
 0.06443425,
 0.010704596,
 -0.035223093,
 0.031429857,
 -0.03455726,
 -0.03295307,
 -0.021002546

In [66]:
sad[0][0]

Record(id=6, payload={'Description': 'Named for its red colored sap that flows from cut stems, bloodroot flowers are white with a bright yellow center. Needs well-drained soil. Blooms in early spring.', 'Id': 'AtfxNCsTNK', 'Name': 'Bloodroot'}, vector=None, shard_key=None)

In [59]:
a = Product(**sad[0][0].payload)

In [63]:
VecProduct()

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [71]:
sad = await client.delete_collection(
    collection_name ="products_description"
)

In [78]:
requests.post("http://127.0.0.1:8000/products/vec_search/", json={'question': 'Red flower'}).json()


{'detail': 'Vector DB error'}

In [77]:
count = await client.count(collection_name="products_description")
count.count

0

In [47]:
# Connect to the MongoDB server
client_mongo = pymongo.MongoClient("mongodb://localhost:27017/")

# Select your database
db = client_mongo["ecommerce-app"]

# List all collection names in the database
collection_names = db.list_collection_names()


collection = db['products']
# Find all documents in the collection
documents = collection.find()
for x in documents:
    print(x)


# print(len(documents))




{'_id': ObjectId('6660a491e6f5fdc838303e16'), 'id': '_ihvY2cxEF', 'name': 'Periwinkle', 'description': 'Perfect as a ground cover, the flowers resemble small, delicate stars and are often found in shades of blue, purple, and white. Needs droughty. Blooms in mid-spring to mid-fall.', 'sell_price': 8.1, 'quantity': 40, 'buy_price': 12.2, 'date': 'None', 'image_id': '6660a491e6f5fdc838303e14', 'tags': ['flower'], 'is_enabled': True}
{'_id': ObjectId('6660a491e6f5fdc838303e19'), 'id': 'X8tcuAiPKf', 'name': 'Bellflower', 'description': 'Known for their resemblance to bells, these flowers come in shades of blue, purple, pink, and white and add a whimsical touch to any garden. Needs well-drained soil. Blooms in early summer to early fall.', 'sell_price': 13.1, 'quantity': 67, 'buy_price': 13.7, 'date': 'None', 'image_id': '6660a491e6f5fdc838303e17', 'tags': ['flower'], 'is_enabled': True}
{'_id': ObjectId('6660a491e6f5fdc838303e1c'), 'id': 'J7hVRECgEm', 'name': 'Lily', 'description': 'A highl

In [ ]:
db.drop_collection('products')
for document in documents:
    print(document)

In [6]:
from pydantic import BaseModel,Field
class Product(BaseModel):
    id: str = Field(default_factory=lambda: nanoid.generate(size=10))
    name: str = ''
    description: str = ''
    sell_price: float = 0
    quantity: int = 0
    buy_price: float = 0
    date: str = Field(default_factory=lambda: datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    image_id: str | None = None
    tags: List[str] = Field(default_factory=list)
    is_enabled: bool = True

class VecProduct(BaseModel):
    id: str = ''
    name: str = ''
    description: str = ''
    is_enabled: bool = True

In [34]:
from sentence_transformers import SentenceTransformer
from qdrant_client import AsyncQdrantClient
from qdrant_client.models import Filter, FieldCondition, PointStruct, MatchValue, PointVectors

class QdrantManager:
    qdrant_client: AsyncQdrantClient
    modelEmbed: SentenceTransformer

    def __init__(self) -> None:
        self.qdrant_client = AsyncQdrantClient('http://127.0.0.1:6333')
        self.modelEmbed = SentenceTransformer('intfloat/e5-small-v2', cache_folder='src/model_st')
        pass

    async def search(self, query:str):
        vector = self.modelEmbed.encode(query, normalize_embeddings=True)
        response = await self.qdrant_client.search(collection_name="products_description",
                                          query_vector=vector,
                                          limit=3,
                                          query_filter= Filter(
                                              must=[
                                                  FieldCondition(
                                                      key='is_enabled',
                                                      match = MatchValue(value=True)
                                                  )
                                              ]
                                          ))
        return response

    async def add_product_vec(self,product: Product):
        if not await self.__check_db__(product.name):
            vector = self.modelEmbed.encode(product.description,normalize_embeddings=True)
            num = await self.qdrant_client.count(collection_name="products_description")
            point = PointStruct(
                        id=int(num.count),
                        vector=vector,
                        payload={"id": product.id, "name": product.name, 
                                 "description": product.description, 
                                 "is_enabled": product.is_enabled}
                    )

            await self.qdrant_client.upsert(
                collection_name="products_description",
                wait=True,
                points=[point])
        else:
            print('Product already exists')

    async def update_product(self,product: Product):
        product_in_base = await self.__check_db__(product.name)
        if product.is_enabled:
            updated_product = VecProduct(**product_in_base.payload)
            updated_product.description = product.description
            vector = self.modelEmbed.encode(product.description,normalize_embeddings=True)
            # point = PointVectors(
            #         id=int(product_in_base.id),
            #         vector=vector,
            #         payload=dict(updated_product)
            #     )
            payload=dict(updated_product)
            await self.qdrant_client.update_vectors(
                collection_name='products_description',
                points = [
                    PointVectors(
                    id=int(product_in_base.id),
                    vector=vector,
                )
                ]
            )
        else:
            updated_product = VecProduct(**product_in_base.payload)
            updated_product.is_enabled = product.is_enabled
            # point = PointVectors(
            #         id=int(product_in_base.id),
            #         vector=product_in_base.vector,
            #         payload=dict(updated_product)
            # )
            payload=dict(updated_product)
        await self.qdrant_client.overwrite_payload(
            collection_name="products_description",
            wait = True,
            points=Filter(must=[FieldCondition(
                            key='name',
                            match = MatchValue(value=updated_product.name)
            )]),
            payload=payload
        )

    async def __check_db__(self,name):
        record = await self.qdrant_client.scroll(
        collection_name ="products_description",
        scroll_filter = Filter(
            must = [FieldCondition(
                key='name',
                match =MatchValue(
                    value= name))
                    ]
                ),
                with_vectors = True
            )
        if record[0]:
            return record[0][0]
        else:
            return False



In [35]:
dwa = QdrantManager()

In [36]:
example = Product()
example.buy_price = 1
example.name = 'Kwiat'
example.description = 'szesc dwa'
example.sell_price = 1
example.buy_price = 1
example.quantity = 5
example.tags = ['dwa']


In [40]:
await dwa.search('szesc')

[ScoredPoint(id=0, version=5, score=0.9322207, payload={'description': 'szesc dwa', 'id': 'ghzCyW6g4q', 'is_enabled': True, 'name': 'Kwiat'}, vector=None, shard_key=None)]

In [38]:
await dwa.add_product_vec(example)

Product already exists


In [39]:
await dwa.update_product(example)

In [135]:
qdrant_client = QdrantClient('localhost:6333')
results = qdrant_client.search(
    collection_name="products_description",
    query_vector=list(model.encode('dwa', normalize_embeddings=True)),
    limit=3,
)
type(results[0])

qdrant_client.http.models.models.ScoredPoint

In [52]:
model[1].word_embedding_dimension

384

In [50]:
count = await client.count(collection_name='products_description')

print(count)

count=48


In [44]:
await client.delete_collection(collection_name='products_description')

True

In [23]:
'/dwa/trzy.jpg'.split('.')

['/dwa/trzy', 'jpg']